In [1]:
import copy
import itertools
import os
from typing import List

import matplotlib.pyplot as plt
import pandas as pd
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

from semantic_chunkers import StatisticalChunker
# from semantic_router.encoders import OpenAIEncoder
from src.encoder import OpenAIEncoder

In [3]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="../.env", env_file_encoding="utf-8", extra="ignore"
    )
    embedding_base_url: str
    embedding_api_key: str
    embedding_model: str
    embedding_model_dir: str
    
    sample_data_dir: str
    pipeline_src_dir: str
settings = Settings()

import sys
sys.path.append(settings.pipeline_src_dir)

In [4]:
## Load Sample
df = pd.read_parquet(settings.sample_data_dir)
df = df.sample(100)
print(df.shape, df.columns)

(100, 7) Index(['id', 'title', 'abstract', 'authors', 'published_date', 'link',
       'markdown'],
      dtype='object')


In [5]:
df.head()

,id,title,abstract,authors,published_date,link,markdown
52803,2306.12543,Matroid lifts and representability,A 1965 result of Crapo shows that every elemen...,"Daniel Irving Bernstein, Zach Walsh",2023-06-21T20:16:12Z,http://arxiv.org/abs/2306.12543v1,# Matroid lifts and representability\n\n######...
32340,2303.15305,Why winning a soccer game is worth 5 points,Ranking is used in sport leagues to determine ...,Leszek Szczecinski,2023-02-22T21:10:44Z,http://arxiv.org/abs/2303.15305v1,# Why winning a soccer game is worth 5 points\...
41021,2305.10923,Query Performance Prediction: From Ad-hoc to C...,Query performance prediction (QPP) is a core t...,"Chuan Meng, Negar Arabzadeh, Mohammad Aliannej...",2023-05-18T12:37:01Z,http://arxiv.org/abs/2305.10923v1,# Query Performance Prediction:\n\n###### Abst...
48656,2310.15802,Nonlinear response theory for lossy supercondu...,We introduce a numerically exact and yet compu...,"V. Vadimov, M. Xu, J. T. Stockburger, J. Anker...",2023-10-24T12:53:10Z,http://arxiv.org/abs/2310.15802v1,# Nonlinear response theory for lossy supercon...
17079,2302.04949,Almost-Nash Sequential Bargaining,"In a 2017 paper, later presented at the Web an...","Gokul Dharan, Hunter Guru, Michael Sun",2023-02-09T21:40:41Z,http://arxiv.org/abs/2302.04949v1,# Almost-Nash Sequential Bargaining\n\n###### ...
